## Importations et chargement

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import util
csv_path = 'data/CES19.csv'


In [ ]:
df = pd.read_csv(csv_path, index_col=0)
df.head()

df_columns = df.columns

In [ ]:
samples_per_column = get_samples_per_column()

samples_per_column

### Enlever les colonnes ayant plus de 125 réponses uniques

In [ ]:
unique_answers_max = 125
columns_with_too_many_unique_answers = get_unique_values_per_column(min_unique_threshold=unique_answers_max)

print(f"\nIl y a {len(columns_with_too_many_unique_answers)} colonnes avec trop de réponses uniques.\n")

# Create another df without those columns
df2 = df.loc[:, [column not in columns_with_too_many_unique_answers for column in df.columns]]

### Enlever les colonnes ayant 0 ou 1 réponse unique

In [ ]:
unique_answers_min = 2
columns_without_enough_unique_answers = get_unique_values_per_column(max_unique_threshold=unique_answers_min)

print(f"\nIl y a {len(columns_without_enough_unique_answers)} colonnes avec trop peu de réponses uniques.\n")

# Create another df without those columns
df3 = df2.loc[:, [column not in columns_without_enough_unique_answers for column in df2.columns]]

### Enlever les questions jugées non-pertinentes

In [ ]:
non_pertinent_columns = [
    "cps19_consent", # La question de consentement à répondre avant de commencer le questionnaire
    
]

print(f"\nIl y a {len(non_pertinent_columns)} colonnes non pertinentes.\n")

df4 = df3.loc[:, [column not in non_pertinent_columns for column in df3.columns]]

df4

In [ ]:
df4

df[["pes10_socnet3", "pes19_keepromises"]]

### Trouver les questions à choix multiples non étalées sur plusieurs questions

In [ ]:
multiple_choice_question_keywords = ["Very satisfied", "None of these", "Got better", "Fewer immigrants", "Fewer refugees", "Not at all well", "Very opposed", "Fairly strongly", "Some of the time", "Much closer", "A good job", "None at all", "Very comfortable", "Extremely easy", "Weekly", "Sometimes", "Just once", "Much more"]
#print(get_columns_per_value_keyword("Very satisfied"))

# Questions à 7 choix
six_choice_questions = ['pes10_socnet3']
print(f"Il y a {len(six_choice_questions)} questions à 7 choix.")

# Questions à 6 choix
six_choice_questions = ['pes19_keepromises', 'pes19_tieus', 'pes19_tiechina', 'pes19_emb_info', 'pes19_donerm', 'pes19_donew', 'pes19_donegl', 'pes19_doneqc', 'pes19_taxes_1', 'pes19_taxes_2', 'pes19_taxes_3', 'pes19_taxes_4', 'pes19_taxes_5', 'pes19_gap']
print(f"Il y a {len(six_choice_questions)} questions à 6 choix.")

# Questions à 5 choix
five_choice_questions = ['cps19_demsat', 'cps19_fed_gov_sat', 'cps19_prov_gov_sat', 'pes19_dem_sat', 'pes19_emb_satif', "cps19_rel_imp", 'cps19_pos_fptp', 'cps19_pos_life', 'cps19_pos_cannabis', 'cps19_pos_carbon', 'cps19_pos_energy', 'cps19_pos_envreg', 'cps19_pos_jobs', 'cps19_pos_subsid', 'cps19_pos_trade', 'cps19_govt_confusing', 'cps19_govt_say', 'cps19_pol_eth', 'cps19_lib_promises', 'pes19_pos_fptp', 'pes19_paymed', 'pes19_senate', 'pes19_envirojob', 'pes19_hatespeech', 'pes19_losetouch', 'pes19_womenhome', 'pes19_govtcare', 'pes19_complicated', 'pes19_famvalues', 'pes19_pollie', 'pes19_bilingualism', 'pes19_equalrights', 'pes19_ethid', 'pes19_fitin', 'pes19_immigjobs', 'pes19_govteff', 'pes19_govtprograms', 'pes19_emb_none', 'pes19_emb_id', 'pes19_emb_vote16', 'pes19_lowturnout', 'pes19_internetvote1', 'pes19_mediaelite', 'pes19_medianolie', 'pes19_opinion', 'pes19_lookslikeme', 'pes19_womenparl', 'pes19_populism_1', 'pes19_populism_2', 'pes19_populism_3', 'pes19_populism_4', 'pes19_populism_5', 'pes19_populism_6', 'pes19_populism_7', 'pes19_populism_8', 'pes19_nativism1', 'pes19_nativism2', 'pes19_nativism3', 'pes19_nativism4', 'pes19_nativism5', 'pes19_canid1', 'pes19_canid2', 'pes19_canid3', 'pes19_sdo1', 'pes19_sdo2', 'pes19_sdo3', 'pes19_sdo4', 'pes19_abort3', 'pes19_abort6', 'pes19_trade', 'pes19_privjobs', 'pes19_govt_act_ineq', 'pes19_deserve1', 'pes19_deserve2', 'pes19_blame', 'pes19_hostile1', 'pes19_hostile2', 'pes19_hostile3', 'pes19_benevolent1', 'pes19_benevolent2', 'pes19_benevolent3', 'pes19_pos_carbon', 'pes19_pos_energy', 'pes19_newerlife', 'pes19_happy', 'pes19_satisfied', 'pes19_cognition', 'cps19_snclav', 'cps19_quebec_sov', 'pes19_ottawa_perf', 'pes19_conf_inst1_1', 'pes19_conf_inst1_2', 'pes19_conf_inst1_3', 'pes19_conf_inst2_1', 'pes19_conf_inst2_2', 'pes19_conf_inst2_3', 'pes19_conf_inst2_4', 'pes19_conf_inst2_5', 'pes19_conf_inst2_6', 'pes19_conf_inst2_7', 'pes19_conf_inst2_8', 'pes19_conf_inst2_9' 'pes19_internetregis', 'pes19_disagreed', 'pes19_affective_1', 'pes19_affective_2', 'pes19_affective_3', 'pes19_affective_4' 'cps19_volunteer', 'pes19_partic1_1', 'pes19_partic1_2', 'pes19_partic1_3', 'pes19_partic2_1', 'pes19_partic2_2', 'pes19_partic2_3', 'pes19_partic2_4', 'pes19_partic3_1', 'pes19_partic3_2', 'pes19_partic3_3', 'pes19_partic3_4', 'pes19_partic3_5', 'pes19_partic3_6']
print(f"Il y a {len(five_choice_questions)} questions à 5 choix.")

# Questions à 3 choix + Extra (Je ne sais pas/préfère ne pas répondre)
four_choice_questions = ['cps19_spend_educ', 'cps19_spend_env', 'cps19_spend_just_law', 'cps19_spend_defence', 'cps19_spend_imm_min' 'cps19_econ_retro', 'cps19_own_fin_retro', 'cps19_imm', 'cps19_refugees', 'cps19_fed_id_str', 'cps19_prov_id_str', 'pes19_pidtradstrong']
print(f"Il y a {len(four_choice_questions)} questions à 3 choix.")


In [ ]:
def get_normalized_ordinal_question_answers(df, answer_order_dic, question_of_interest, lower_bound=-1, upper_bound=1):
    answers = df[question_of_interest]
    unique_answers_sorted = answers.unique()[answer_order_dic[question_of_interest]]
    answer_to_int = {answer: i for i, answer in enumerate(unique_answers_sorted)}

    # Replace each answer with its corresponding integer
    normalized_answers = answers.map(answer_to_int)

    # Normalize the answers
    return normalize_answers(normalized_answers, lower_bound, upper_bound)

def normalize_answers(answers, lower_bound=-1, upper_bound=1):
    # Normalize the answers between lower_bound and upper_bound
    normalized_answers = (answers - answers.min()) / (answers.max() - answers.min()) * (upper_bound - lower_bound) + lower_bound

    return normalized_answers

answer_order_dic = {
    'cps19_demsat': [4, 3, 1, 0, 2]
}

question_of_interest = 'cps19_demsat'
get_normalized_ordinal_question_answers(df, answer_order_dic, question_of_interest)

In [ ]:
col_of_interest = "cps19_citizenship"
print(df[col_of_interest].value_counts())
plot_column_values(col_of_interest)

### Isoler les questions "étiquette"

In [ ]:
label_questions = ['cps19_votechoice', 'cps19_votechoice_pr', 'cps19_vote_unlikely', 'cps19_vote_unlike_pr', 'cps19_v_advance']
label_detail_questions = ['cps19_votechoice_7_TEXT', 'cps19_votechoice_pr_7_TEXT', 'cps19_vote_unlikely_7_TEXT', 'cps19_vote_unlike_pr_7_TEXT', 'cps19_v_advance_7_TEXT', 'cps19_vote_lean', 'cps19_vote_lean_7_TEXT', 'cps19_vote_lean_pr', 'cps19_vote_lean_pr_7_TEXT', 'cps19_2nd_choice', 'cps19_2nd_choice_7_TEXT', 'cps19_2nd_choice_pr', 'cps19_2nd_choice_pr_7_TEXT', 'cps19_not_vote_for_1', 'cps19_not_vote_for_2', 'cps19_not_vote_for_3', 'cps19_not_vote_for_4', 'cps19_not_vote_for_5', 'cps19_not_vote_for_6', 'cps19_not_vote_for_7', 'cps19_not_vote_for_8', 'cps19_not_vote_for_9', 'cps19_not_vote_for_7_TEXT']

### Liste metadata pertinente/non-pertinente

In [ ]:
meta_data_pertinente = ["cps19_current_date",
                        "cps19_Q_Language",         #lanque de reponse du repondant
                        "cps19_data_quality",       # certaines indication sur qualité data
                        "cps19_age",
                        "cps19_duplicates_flag",
                        "cps19_inattentive",        # peut donner un hint sur qualité des données 1 si innatentif
                        "pes19_current_date",
                        "pes19_Q_Language",
                        "pes19_age",
                        "pes19_duplicates_flag",
                        "pes19_inattentive",

                        ]



meta_data_non_pertinente = ["cps19_StartDate",
                            "cps19_EndDate",
                            "cps19_ResponseId",
                            "cps19_current_date_string",    #dupliqué de cps19_current_date
                            "cps19_Q_TotalDuration",
                            "get_news",                 # 1 si repondant fait poser la question cps19_news_cons
                            "get_more_naming",          # c'est fait poser cps19_govgen_name et cps19_presrus_name
                            "get_not_vote_for",
                            "get_party_issue_handling",
                            "get_imp_loc_iss",
                            "get_outcome",
                            "justice_law",
                            "justice_law_fr",           # différence mineure dans le wording de la question
                            "lr_scale_order",
                            "ethnicity_intro",          # différence mineure de question en fonction de citizenship
                            "ethnicity_intro_fr",
                            "premier",
                            "province_fr",
                            "pid_en",
                            "pid_party_en",
                            "cps19_data_quality",
                            "cps19_panel",              # "Which panel the respondent was drawn from"
                            "constituencynumber",       # numero de circonscription ne semble pas surper pertinent
                            "constituencyname",
                            "cps19_fsa",
                            "cps19_weight_general_all", #poids associé au répondant
                            "cps19_weight_general_restricted",
                            "pes19_StartDate",
                            "pes19_EndDate",
                            "pes19_ResponseId",
                            "pes19_current_date_string",
                            "pes19_Q_TotalDuration",
                            "pes19_fsa",
                            "notvote_split",
                            "splitsample",
                            "confidence_institutions_word",
                            "confidence_institutions_word_fr",
                            "govt_programs_word",
                            "govt_programs_word_fr",
                            "split_taxes",
                            "split_senate",
                            "split_trade",
                            "split_lifesat",
                            "split_responsibility",
                            "split_sexism",
                            "split_abortion",
                            "split_getahead",
                            "split_att_div",
                            "split_govt_eff",
                            "split_medical",
                            "split_ties",
                            "split_health_followups",
                            "split_gender_id",
                            "split_big5",
                            "split_hatespeech",
                            "split_vol_assoc",
                            "PES19_data_quality",
                            "pes19_panel",
                            "pes19_weight_general_all",
                            "pes19_weight_general_restricted"]


### Querstions pertinentes/non-pertientes

In [ ]:
questions_pertinentes = [
    "cps19_citizenship",
    "cps19_gender",         # string "A man" "A woman" "Other (e.g. Trans, non-binary, two-spirit, gender-queer)"
    "cps19_province",
    "cps19_education",
    "cps19_demsat",
    "cps19_imp_iss_party",  # personnes qui n'ont pas répondue à cps19_imp_iss n'ont pas la question
    "cps19_imp_loc_iss_p",
    "cps19_votechoice",
    "cps19_votechoice_pr",  # pas vraiment certain de celle là. S'addresse aux pr qui ne peuvent pas voter
    "cps19_vote_unlikely",  # idem
    "cps19_vote_unlike_pr", #idem(p-etre regrouper avec cps19_votechoice_pr cps19_vote_unlikely cps19_votechoice
    "cps19_v_advance",      # vote par anticipation (s'ils ont déjà voté, pour qui ils ont voté)
    "cps19_vote_lean",      # regrouper avec cps19_vote_unlike_pr et autre
    "cps19_vote_lean_pr",   # regrouper avec cps19_vote_unlike_pr et autre
    "cps19_2nd_choice",     #
    'cps19_2nd_choice_pr',  # merge avec cps19_2nd_choice
    "cps19_not_vote_for",   # (_1 - _9)
    "cps19_fed_gov_sat",    # deg de satisfaction du gouv libéral
    "cps19_lr_scale_bef_1",     # slider pour gauche vs droite. 0=left, 10 = droite
    "cps19_lr_parties",         # slider gauche vs droite party. 0=left, 10 = droite
    "cps19_party_rating",   # Sliders (merge _23 - _28)?
    "cps19_lead_rating",    # Sliders (merge _23 - _28)?
    "cps19_cand_rating_23", # Sliders (merge _23 - _28)? par certain, spécifique au candidat dans la circonscription
    "cps19_lead_int_113",   # Justin Trudeau (113)
    "cps19_lead_int_114",   # Andrew Scheer (114)
    "cps19_lead_int_115",   # Jagmeet Singh (115)
    "cps19_lead_int_116",   # Yves-François Blanchet (116)
    "cps19_lead_int_117",   # Elizabeth May (117)
    "cps19_lead_int_118",   # Maxime Bernier (118)
    "cps19_lead_strong_113",# Justin Trudeau (113)
    "cps19_lead_strong_114",# Andrew Scheer (114)
    "cps19_lead_strong_115",# Jagmeet Singh (115)
    "cps19_lead_strong_116",# Yves-François Blanchet (116)
    "cps19_lead_strong_117",# Elizabeth May (117)
    "cps19_lead_strong_118",# Maxime Bernier (118)
    "cps19_lead_trust_113", # Justin Trudeau (113)
    "cps19_lead_trust_114", # Andrew Scheer (114)
    "cps19_lead_trust_115", # Jagmeet Singh (115)
    "cps19_lead_trust_116", # Yves-François Blanchet (116)
    "cps19_lead_trust_117", # Elizabeth May (117)
    "cps19_lead_trust_118", # Maxime Bernier (118)
    "cps19_lead_cares_113", # Justin Trudeau (113)
    "cps19_lead_cares_114", # Andrew Scheer (114)
    "cps19_lead_cares_115", # Jagmeet Singh (115)
    "cps19_lead_cares_116", # Yves-François Blanchet (116)
    "cps19_lead_cares_117", # Elizabeth May (117)
    "cps19_lead_cares_118", # Maxime Bernier (118)
    "cps19_spend_educ",     # opinion priorité, renseigne sur alignement politique éducation
    "cps19_spend_env",      # opinion priorité, renseigne sur alignement politique environement
    "cps19_spend_just_law", # opinion priorité, renseigne sur alignement politique systeme de justice
    "cps19_spend_defence",  # opinion priorité, renseigne sur alignement politique armée
    "cps19_spend_imm_min",  # opinion priorité, renseigne sur alignement politique immigration
    "cps19_pos_cannabis",   # opinion priorité, renseigne sur alignement politique criminalisation de la posession de cannabis
    "cps19_pos_carbon",     # merge possible avec cps19_spend_env
    "cps19_pos_energy",     # opinion, alligmeent politique secteur energie petrole (merge possible cps19_spend_env)
    "cps19_pos_envreg",     # opinion, alignement politique sur regulation environnement. (merge possible cps19_spend_env)
    "cps19_pos_jobs",       # opinion, alignement politique sur prio environnement vs jobs. (merge possible cps19_spend_env)
    "cps19_pos_subsid",     # opinion, alignement politique sur aide financière aux entreprises.
    "cps19_econ_retro",     # opinion, economie canadienne s'est améliorée ou pas
    "cps19_econ_fed",       # merge possible avec cps19_econ_retro
    "cps19_ownfinanc_fed",  # merge possible avec cps19_econ_retro
    "cps19_issue_handle",   # multiple dimensions question difficile à analyser (need merge)
    "cps19_imm",            # maybe merge avec cps19_spend_imm_min
    "cps19_refugees",       # maybe merge avec cps19_spend_imm_min
    "cps19_lib_promises",   #indicateur de satsifaction avec statut quo
    "cps19_quebec_sov",     # souveraineté du quebec
    "cps19_own_fin_retro",  # relative finance perso, possible merge avec cps19_econ_retro (maybe)
    "cps19_fed_id",         # question directe d'affiliation politique
    "cps19_fed_id_str",     # niveau de confiance dans l'affiliation politique federale
    "cps19_prov_id",        # affiliation politique provinciale
    "cps19_prov_id_str",    # niveau de confiance dans l'affiliation politique provinciale
    "cps19_groups_therm",   # (_1-_5) niveau d'appréciation des groupe marginaux
    "cps19_vote_2015",      # quel parti le repondant a voté en 2015
    "cps19_religion",       # religion du repondant. (pas certain pour la pertinence) _1-_23 + text answer.
    "cps19_rel_imp",        # importance de la religion. (maybe merge avec cps19_religion)
    "cps19_bornin_canada",  # peut etre pas super pertinent avec cps19_citizenship. faut examiner les réponses.
    "cps19_bornin_other",   # pays de naissance si "cps19_bornin_canada = non"
    "cps19_employment",     # type d'emploi
    "cps19_sector",         # secteur d'emploi
    "cps19_union",          # fait partie d'un syndicat
    "cps19_children",       # avez vous des enfants.
    "cps19_income_number",  # revenu du repondant
    "cps19_income_cat",     # revenu de la famille du repondant
    "cps19_property",       # avez vous un propriété
    "cps19_marital",        # statut marital
    "cps19_household",      # nombre d'occupant dans l'habitation
    "pes19_province",       # province de residence (maybe merge avec cps19_province)
    "pes19_citizen",        # maybe merge avec cps19
    "pes19_votechoice2019", # ******choix de vote*******
    "pes19_pr_votechoice",  # ******choix de vote******* (pour ceux qui n'ont pas pu voter) merge avec pes19_votechoice2019
    "pes19_groups1",        #  _1-_4 niveau d'appréciation des groupe marginaux
    "pes19_econ_retro",     # merge avec cps19_econ_retro
    "pes19_pos_fptp",       # changer mode scrutin
    "pes19_paymed",         # opinion sur systeme medical a deux vitesses
    "pes19_senate",         # opininon sur sénat
    "pes19_envirojob",      # merge avec cps19_pos_jobs
    "pes19_hatespeech",     # opinion sur discours haineux
    "pes19_losetouch",      # opinion sur perte de contact des élus avec la population
    "pes19_womenhome",      # opinion sur travail des femmes
    "pes19_govtcare",       # le gouvernement ne trouve pas important ce que des personnes comme moi pensent
    "pes19_bilingualism",   # bilinguisme, trop ou pas assez?
    "pes19_equalrights",
    "pes19_fitin",          # peut etre merge avec cps19_imm
    "pes19_immigjobs",      # peut etre merge avec cps19_imm
    "pes19_tieus",          # lien avec les US
    "pes19_tiechina",       # lien avec la chine
    "pes19_country",        # sliders (_1-_4)peux donner de l'info
    "pes19_ottawa_perf",    # performance du gouv liberal dernier 4 ans
    "pes19_party_rep_whic", # parti qui représente le mieux ma vision
    "pes19_party_rate",     # (_10_14) sliders sur appréciation des partis
    "pes19_lead_rate",      # (_7-_11) sliders sur appréciation des leaders
    "pes19_lr_parties",     # (_1-_5) left-right parties
    "pes19_lr_self",        # left_right repondant
    "pes19_emb_none",       # pourrait permettre de classifier "dont know"
    "pes19_conf_inst1",     # (_1-_3) confiance en certaines institutions (va être difficile à exploiter)
    "pes19_conf_inst2",     # (_1-_9) confiance en certaines institutions (va être difficile à exploiter)
    "pes19_provvote",       # intention vote election provinciales
    "pes19_volassoc",       # implication dans organisation spécifiques
    "pes19_lookslikeme",    # représentation par meme ethnicité
    "pes19_womenparl",      # representation par plus de femmes
    "pes19_corruption",     # corruption politiciens, peut discriminé lib. cons. vs les autres
    "pes19_populism_6",     # peut discriminé avec la réponse d'opinion au leader pes19_lead_rate
    "pes19_populism_7",     # peut discriminé centre-droite et droite
    "pes19_nativism3",      # immigrants bon pour economie
    "pes19_nativism4",      # immigrants pas bons pour culture
    "pes19_nativism5",      # immigrants criminels
    "pes19_sdo1",           # certain groupes devrait rester chez eux
    "pes19_sdo2",           # egalité des groupes
    "pes19_sdo3",           # merge avec pes19_sdo2
    "pes19_sdo4",           # etes vous supremaciste?
    "pes19_donerm",         # aide aux minorités
    "pes19_donew",          # aide aux femmes
    "pes19_donegl",         # aide aux homosexuels
    "pes19_doneqc",         # aide au Quebec (??)
    "pes19_taxes",          #(_1-_5) taxer plus differents groupes
    "pes19_abort1",         # opinions avortement
    "pes19_abort2",         # merge avec pes19_abort1
    "pes19_abort3",         # merge avec pes19_abort1
    "pes19_abort4",         # merge avec pes19_abort1
    "pes19_abort5",         # merge avec pes19_abort1
    "pes19_abort6",         # merge avec pes19_abort1
    "pes19_trade",          # trade_intl vs jobs
    "pes19_privjobs",       # gouv vs privé influence sur jobs
    "pes19_govt_act_ineq",  # gouv + d'influence sur inegalite salariales
    "pes19_deserve1",       # personnes pas emplyées le mérite
    "pes19_deserve2",       # opinion sur aide sociale
    "pes19_blame",          # opinion sur autonomie personnelle et niveau social
    "pes19_stdofliving",    # merge avec pes19_deserve2
    "pes19_inequal",        # opinion inegalite sociale
    "pes19_gap",            # influence gouv sur inegalite sociale
    "pes19_provfed",        # opinion sur grosseur gouv.
    "pes19_hostile1",       # etes vous misogine, merge potentiel avec pes19_done
    "pes19_hostile2",       # merge pes19_hostile1
    "pes19_hostile3",       # merge pes19_hostile1
    "pes19_benevolent1",    # merge pes19_hostile1
    "pes19_benevolent2",    # merge pes19_hostile1
    "pes19_benevolent3",    # merge pes19_hostile1
    "pes19_cc1",            # changement climatique
    "pes19_pid",            # merge avec pes19_party_rep_whic
    "pes19_pidtrad",        # merge avec pes19_party_rep_whic
    "pes19_langQC",         # identification BQ
    "pes19_cultureQC",      # merge avec pes19_langQC
    "pes19_qclang",         # merge avec pes19_langQC
    "pes19_newerlife",      # filtre progressisme
    "pes19_feminine",       # female gender identity (faudrait faire du bining)
    "pes19_masculine",      # male gender identity (faudrait faire du bining)
    "pes19_service_freq",   # frequence religion
    "pes19_parents_born",   # parents immigrants
    "pes19_rural_urban",    # residence rurale ou ville
    "pes19_lang",           # langque parlée
]

# NA si pas intelligent, nom du candidat si intelligent
# Justin Trudeau (113)
# Andrew Scheer (114)
# Jagmeet Singh (115)
# Yves-François Blanchet (116)
# Elizabeth May (117)
# Maxime Bernier (118)
# Aucun de ces chefs(119)
# Je ne sais pas/Préfère ne pas répondre(120)


df["cps19_lead_int_113"] = df["cps19_lead_int_113"].notna()
df["cps19_lead_int_114"] = df["cps19_lead_int_114"].notna()
df["cps19_lead_int_115"] = df["cps19_lead_int_115"].notna()
df["cps19_lead_int_116"] = df["cps19_lead_int_116"].notna()
df["cps19_lead_int_117"] = df["cps19_lead_int_117"].notna()
df["cps19_lead_int_118"] = df["cps19_lead_int_118"].notna()
df["cps19_lead_strong_113"] = df["cps19_lead_strong_113"].notna()
df["cps19_lead_strong_114"] = df["cps19_lead_strong_114"].notna()
df["cps19_lead_strong_115"] = df["cps19_lead_strong_115"].notna()
df["cps19_lead_strong_116"] = df["cps19_lead_strong_116"].notna()
df["cps19_lead_strong_117"] = df["cps19_lead_strong_117"].notna()
df["cps19_lead_strong_118"] = df["cps19_lead_strong_118"].notna()
df["cps19_lead_trust_113"] = df["cps19_lead_trust_113"].notna()
df["cps19_lead_trust_114"] = df["cps19_lead_trust_114"].notna()
df["cps19_lead_trust_115"] = df["cps19_lead_trust_115"].notna()
df["cps19_lead_trust_116"] = df["cps19_lead_trust_116"].notna()
df["cps19_lead_trust_117"] = df["cps19_lead_trust_117"].notna()
df["cps19_lead_trust_118"] = df["cps19_lead_trust_118"].notna()
df["cps19_lead_cares_113"] = df["cps19_lead_cares_113"].notna()
df["cps19_lead_cares_114"] = df["cps19_lead_cares_114"].notna()
df["cps19_lead_cares_115"] = df["cps19_lead_cares_115"].notna()
df["cps19_lead_cares_116"] = df["cps19_lead_cares_116"].notna()
df["cps19_lead_cares_117"] = df["cps19_lead_cares_117"].notna()
df["cps19_lead_cares_118"] = df["cps19_lead_cares_118"].notna()
# list(df.columns)


In [ ]:
questions_non_pertinentes = [
    "cps19_consent",
    "cps19_yob",                # Dupliqué de cps19_age
    "cps19_yob_2001_age",
    "cps19_imp_iss",            # enjeux le plus important (text based answers)... cps19_imp_iss_party plus relevant
    "cps19_imp_loc_iss",        # enjeux local, les réponses vont surement trop varier. (plein de nans aussi)
    "cps19_interest_gen",       # peut être un bon indicateur de l'intention de voter, mais on s'en fou un peu
    "cps19_interest_elxn",      # idem
    "cps19_v_likely",           # idem
    "cps19_v_likely_pr",        # idem
    "cps19_lr_scale_aft",       # cps19_lr_scale_bef_1 mais la question est posé arpès cps19_lr_parties (check de cohérence)
    "cps19_pos_fptp",           # mode de scrutin
    "cps19_pos_life",           # aide médical à mourrir
    "cps19_pos_trade",          # opinion, alignement politique sur libre marché (pas certain qu'on peu discriminé avec ça)
    "cps19_most_seats",         # pas certain que l'estimé du nombre de députés est relevent
    "cps19_win_local",          # même chose
    "cps19_outcome_most",       # outcome espéré risque d'être vraiment bruité. (text based answer en plus)
    "cps19_outcome_least",      # même chose (text based answer en plus)
    "cps19_govt_confusing",     # parle du niveau de compréhension du systeme politique. pas tant relevant
    "cps19_govt_say",           # parle du niveau d'impuissance vis à vis le systeme politique. pas tant relevant
    "cps19_pol_eth",            # opinion: est-ce que les politiciens agissent éthiquement. pas tant relevant
    "cps19_snclav",             # trop spécific
    "cps19_news_cons",          # evalue le niveau de politisation du repondant
    "cps19_volunteer",          # niveau d'implication sociale
    "cps19_duty_choice",        # vote = devoir ou choix
    "cps19_presrus_name",       # data_filter
    "get_news",                 # data filter
    "get_outcome",              # data filter
    "get_party_issue_handling", # data filter
    "cps19_premier_name",       # test de politisation (nom du 1er ministre prov.)
    "cps19_finmin_name",        # test de politisation (nom du ministre des finances)
    "cps19_govgen_name",        # test de politisation (nom du gouv. générale du canada)
    "cps19_presrus_name",       # test de politisation (nom du president russe)
    "cps19_prov_gov_sat",       # niveau de statisfaction premier ministre gouv
    "cps19_party_member",       # demande si membre d'un parti (non-spécifique). ne va pas donner d'info sur beaucoup de monde
    "cps19_fed_member",         # demande quel de quel parti fed le repondant est membre.
    "cps19_prov_member",        # demande quel de quel parti prov le repondant est membre.
    "cps19_fed_donate",         # donnateur d'un parti fed après 2015
    "cps19_spoil",              # si le repondant a deja annulé son vote.
    "cps19_turnout_2015",       # si le repondant a voté en 2015
    "cps19_debate_en",          # si le repondant a regardé le débat englais
    "cps19_debate_fr",          # si le repondant a regarde le debat francais
    "cps19_imm_year",           # année d'immigration (commence a etre pas mal pointu comme question)
    "cps19_ethnicity",          # ethnicité
    "cps19_sexuality",          # orrientation sexuelle
    "cps19_language",           # langue maternelle
    "pes19_consent",            # consent
    "pes19_mostimpissue",       # enjeux le plus important (text based answers)...
    "pes19_turnout2019",        # pas important si le repondant a pas voté
    "pes19_turnout2019_v2",     # dupliqué
    "pes19_notvotereason1",     # raison du non-vote
    "pes19_notvotereason2",     # dupliqué
    "pes19_howvote",            # manière du vote
    "pes19_votereason",         # raison du vote
    "pes19_dem_sat",            # satisfaction systeme politique
    "pes19_campatt",            # niveau d'attention à la campagne
    "pes19_contact1",           # est-ce qu'un parti a contacté le repondant
    "pes19_contact2",           # est-ce qu'un parti a contacté le repondant
    "pes19_mandate",            # est-ce que le mandat du gagnant est légitime ?
    "pes19_formgovt",           # opinion sur systeme democratique
    "pes19_keepromises",        # opinion sur fiabilité des partis
    "pes19_libwords",           # mot décrivant le parti libéral. toutes les réponses sont du texte
    "pes19_conwords",           # mot décrivant le parti concervateur. toutes les réponses sont du texte
    "pes19_ndpwords",           # mot décrivant le parti npd. toutes les réponses sont du texte
    "pes19_bqwords",            # mot décrivant le parti quebecois. toutes les réponses sont du texte
    "pes19_greenwords",         # mot décrivant le parti vert. toutes les réponses sont du texte
    "pes19_complicated",        # la politique est trop compliquee pour moi
    "pes19_famvalues",          # importance des valeur familliales traditionnelles (ultra biaisé)
    "pes19_pollie",             # est-ce que les politiciens mentent? (non discriminant)
    "pes19_ethid",              # importance de l'ethnicité dans l'identité du répondant.
    "split_govt_eff",           # data_filter
    "pes19_govteff",            # les gouv étaient plus efficaces avant
    "pes19_govtprograms",       # question lié sur la réponse d'une autre question. Difficle d'interprété ça (high hanging fruit)
    "split_ties",               # data filter
    "pes19_can_id",             # (_1-_4) ce qui fait un vrai canadien (pas super important je crois)
    "pes19_party_rep",          # pas vrmt imp sur intention de vote
    "pes19_emb_id",             # pas vraiment important
    "pes19_emb_vote16",         # idem
    "pes19_lowturnout",         # idem
    "pes19_internetvote1",      # idem
    "pes19_internetvote2",      # idem
    "pes19_foreign",            # idem
    "confidence_institutions_word", # data_filter
    "pes19_emb_satif",          # satisfaction enver elections canada
    "pes19_emb8",               # satisfaction enver elections canada
    "pes19_internetregis",      # enregistrement internet elections
    "pes19_internetrisk1",      # election internet risqué
    "pes19_internetrisk2",      # election internet risqué
    "pes19_emb_register",       # recu cart d'enrg par mail
    "pes19_emb_card",           # info carte de vote ok?
    "pes19_emb_register2",      # enrg. durant elections?
    "pes19_emb_reg_how",        # comment enrg.?
    "pes19_emb_register3",      # enrg. difficile?
    "pes19_emb4",               # source d'info elections (text based)
    "pes19_emb7",               # (_2,_3,_5) repondant bien informé?
    "pes19_emb_info",           # facilité d'obtention d'info
    "pes19_interest",           # intéret en politique
    "pes19_socnet1",            # nombre d'amis
    "pes10_socnet3",            # frequence de rencontre avec amis
    "pes19_socnet2",            # (_1-_3) nombre d'amis variés
    "pes19_discfam",            # evaluation du niveau de politisation famille
    "pes19_discwork",           # evaluation du niveau de politisation travail
    "pes19_disagreed",          # niveau de confrontation politique
    "pes19_partic1",            # non discriminant
    "pes19_partic2",            # non discriminant
    "pes19_partic3",            # non discriminant
    "pes19_partymember",        # non discriminant  (repondant membre d'un parti)
    "pes19_diff_power",         # non discriminant
    "pes19_diff_happens",       # non discriminant
    "pes19_mediaelite",         # media distrust
    "pes19_medianolie",         # media trust
    "pes19_opinion",            # non discriminant opinion vs fait
    "pes19_populism_1",         # non discriminant You feel you understand the most important political issues of this country.
    "pes19_populism_2",         # non discriminant En politique, les compromis équivalent à renoncer à ses principes.
    "pes19_populism_3",         # non discriminant Most politicians do not care about the people.
    "pes19_populism_4",         # non discriminant politiciens fiables
    "pes19_populism_5",         # non discriminant politiciens sont le problème
    "pes19_populism_8",         # non discriminant gouv care just pour les riches
    "pes19_nativism1",          # imigrant devraient s'adapter. difficiles a dire, mais je penche un peu vers pas important
    "pes19_nativism2",          # pas vraiment discriminant.
    "pes19_canid1",             # répondant ressemble a d'autres candaiens
    "pes19_canid2",             # identification comme étant un canadien
    "pes19_canid3",             # joi d'etre canadien
    "split_trade",              # data_filter
    "split_getahead",           # data filter
    "pes19_trust",              # méfiance envers les autres
    "split_sexism",             # data filter
    "pes19_cc2",                # pas tant discriminant
    "pes19_pid_close",          # proche d'un parti
    "pes19_little_close",       # un peu proche d'un parti
    "pes19_pid_strength",       # party identification strenght. pas vraiment d'info aditionelle
    "pes19_pidtradstrong",      # party identification strenght. pas vraiment d'info aditionelle
    "pes19_affective",          # (_1-_4) TROP SPÉCIFIQUE
    "pes19_qcsol",              # va ajouter du bruit
    "split_lifesat",            # data_filter
    "pes19_happy",              # satisfaction de la vie
    "pes19_satisfied",          # idem
    "pes19_cognition",          # evaluation cognitive du repondant
    "split_gender_id",          # Data filter
    "pes19_big5",               # evaluation cognitive du repondant
    "pes19_health",             # evaluation santé du repondant
    "split_health_followups",   # data filter
    "split_big5",               # data filter
    "pes19_phealth",            # evaluation santé relative du repondant
    "pes19_mhealth",            # evaluation santé mentale relative du repondant
    "pes19_yob",                # dupliqué pes19_age
    "pes19_month_of_birth",     # mois de naissance
    "pes19_lived",              # cmb d'annees dans communauté
    "pes19_follow_pol",         # suivre nouvelles politique?
    "pes19_occ_text",           # text based answer pas mal bruitée
    "pes19_occ_cat",            # semi text based answer pas mal bruitée

]